<a href="https://colab.research.google.com/github/CubatLin/Data-Science-Project/blob/main/NER%20Tagging/NER_CKIPtagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## To Do:
* 換IP位置爬
* 換成CKIP tagger(可以自訂義辭典的方式)

In [1]:
pip install -U ckiptagger[tf,gdown]

In [2]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

In [3]:
data_utils.download_data_gdown("./") # gdrive-ckip

Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /content/data.zip
100%|██████████| 1.88G/1.88G [00:16<00:00, 117MB/s] 


In [4]:
# To use GPU:
#    1. Install tensorflow-gpu (see Installation)
#    2. Set CUDA_VISIBLE_DEVICES environment variable, e.g. os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#    3. Set disable_cuda=False, e.g. ws = WS("./data", disable_cuda=False)
# To use CPU:
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")

/usr/local/lib/python3.7/dist-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:988: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  partitioner=maybe_partitioner)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:996: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=initializer)
/usr/local/lib/python3.7/dist-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class

In [5]:
article = ['Mar 7, 2019 — 全臺最「網美」的書店，港都特色處處都是打卡熱點 · 國境之南的據點'
       ,'誠品以「人文、藝術、創意、生活」為核心價值，由推廣閱讀出發，並透過線上網路，傳遞博雅的溫度，打造全新的文化場域。']

In [8]:
# 定義字典
word_to_weight = {
    "網美": 1000,"國境之南": 10000,"公有": 2}
dictionary = construct_dictionary(word_to_weight)
print(dictionary)

[(2, {'網美': 1000.0, '公有': 2.0}), (4, {'國境之南': 10000.0})]


In [9]:
%%time
# CPU times: user 1min 25s, sys: 2.3 s, total: 1min 27s
# Wall time: 46.2 s

word_sentence_list = ws(
    article,
    # sentence_segmentation = True, # To consider delimiters
    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
    coerce_dictionary = dictionary # words in this dictionary are forced
)

pos_sentence_list = pos(word_sentence_list)
entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

CPU times: user 2.27 s, sys: 271 ms, total: 2.55 s
Wall time: 1.45 s


In [10]:
def print_word_pos_sentence(word_sentence, pos_sentence):
    assert len(word_sentence) == len(pos_sentence)
    for word, pos in zip(word_sentence, pos_sentence):
        print(f"{word}({pos})", end="\u3000")
    print()
    return
    
for i, sentence in enumerate(article):
    print()
    print(f"'{sentence}'")
    print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
    for entity in sorted(entity_sentence_list[i]):
        print(entity)


'Mar 7, 2019 — 全臺最「網美」的書店，港都特色處處都是打卡熱點 · 國境之南的據點'
Mar (FW)　7,(Neu)　 (WHITESPACE)　2019(Neu)　 — (FW)　全(Neqa)　臺(Nc)　最(Dfa)　「(PARENTHESISCATEGORY)　網美(VH)　」(PARENTHESISCATEGORY)　的(DE)　書店(Nc)　，(COMMACATEGORY)　港都(Nc)　特色(Na)　處處(D)　都(D)　是(SHI)　打卡(VA)　熱點(Na)　 (WHITESPACE)　·(PERIODCATEGORY)　 (WHITESPACE)　國境之南(Nc)　的(DE)　據點(Na)　
(0, 4, 'ORG', 'Mar ')
(15, 16, 'GPE', '臺')
(25, 27, 'GPE', '港都')
(37, 44, 'LOC', ' · 國境之南')

'誠品以「人文、藝術、創意、生活」為核心價值，由推廣閱讀出發，並透過線上網路，傳遞博雅的溫度，打造全新的文化場域。'
誠品(Nb)　以(P)　「(PARENTHESISCATEGORY)　人文(Na)　、(PAUSECATEGORY)　藝術(Na)　、(PAUSECATEGORY)　創意(Na)　、(PAUSECATEGORY)　生活(Na)　」(PARENTHESISCATEGORY)　為(VG)　核心(Na)　價值(Na)　，(COMMACATEGORY)　由(P)　推廣(VC)　閱讀(VC)　出發(VA)　，(COMMACATEGORY)　並(Cbb)　透過(P)　線上(Nc)　網路(Na)　，(COMMACATEGORY)　傳遞(VD)　博雅(Na)　的(DE)　溫度(Na)　，(COMMACATEGORY)　打造(VC)　全新(VH)　的(DE)　文化(Na)　場域(Nc)　。(PERIODCATEGORY)　


In [ ]:
# 'Mar 7, 2019 — 全臺最「網美」的書店，港都特色處處都是打卡熱點 · 國境之南的據點'
# Mar (FW)　7,(Neu)　 (WHITESPACE)　2019(Neu)　 — (FW)　全(Neqa)　臺(Nc)　最(Dfa)　「(PARENTHESISCATEGORY)　網美(VH)　」(PARENTHESISCATEGORY)　的(DE)　書店(Nc)　，(COMMACATEGORY)　港都(Nc)　特色(Na)　處處(D)　都(D)　是(SHI)　打卡(VA)　熱點(Na)　 (WHITESPACE)　·(PERIODCATEGORY)　 (WHITESPACE)　國境(Nc)　之(DE)　南(Ncd)　的(DE)　據點(Na)　
# (0, 4, 'ORG', 'Mar ')
# (15, 16, 'GPE', '臺')
# (25, 27, 'GPE', '港都')

# '誠品以「人文、藝術、創意、生活」為核心價值，由推廣閱讀出發，並透過線上網路，傳遞博雅的溫度，打造全新的文化場域。'
# 誠品(Nb)　以(P)　「(PARENTHESISCATEGORY)　人文(Na)　、(PAUSECATEGORY)　藝術(Na)　、(PAUSECATEGORY)　創意(Na)　、(PAUSECATEGORY)　生活(Na)　」(PARENTHESISCATEGORY)　為(VG)　核心(Na)　價值(Na)　，(COMMACATEGORY)　由(P)　推廣(VC)　閱讀(VC)　出發(VA)　，(COMMACATEGORY)　並(Cbb)　透過(P)　線上(Nc)　網路(Na)　，(COMMACATEGORY)　傳遞(VD)　博雅(Na)　的(DE)　溫度(Na)　，(COMMACATEGORY)　打造(VC)　全新(VH)　的(DE)　文化(Na)　場域(Nc)　。(PERIODCATEGORY)　